# SVD Based Collaborative Filtering Aloghrithm 

# for Movie Recommendation System

** Pre-define functions to be used **

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix, bmat
from time import time
from os.path import isfile
import matplotlib.pyplot as plt
from numpy.linalg import norm

# The csv processing functions are wrapped in helper.py
from helper import csv_parse, write_submission

%matplotlib inline

In [2]:
def RMS(pred, true):
    rms = np.sqrt(np.sum((pred-true)**2)/len(pred))
    return rms

** Read in the training data and pre-process the data **

In [3]:
# Loading the train data
train_csv_raw = './cil-collab-filtering-2018/data_train.csv'
csv_train = './data_train_post.csv'
if isfile(csv_train):
    print('Read in processed csv_train file: Success!')
    df_train = pd.read_csv(csv_train)
else:
    df_train = csv_parse(train_csv_raw, csv_train)

# df_train.head()

Read in processed csv_train file: Success!


### Step 1: Creat a baseline solution 

We create a baseline solution by setting the missing values to the average over all observed ratings for a particular item. 

In [4]:
# Define the size of the training data
global user_N, item_N 
user_N = 100000
item_N = 1000
    
# Prepare baseline matrix 
# Calculate mean rating for every single item
mean_per_item = df_train.groupby('col_id')['Prediction'].mean().as_matrix()

# Form A with sparse matrix (more efficient)
A = coo_matrix((df_train['Prediction'], 
                (df_train['row_id']-1, df_train['col_id']-1))
              ).todense()
A = A + mean_per_item 
A[df_train['row_id']-1, df_train['col_id']-1] = df_train['Prediction']

# write_submission(A, dst='./cil-collab-filtering-2018/submission_baseline.csv')

### Step 2: SVD Decomposition

Compute the SVD Decompostion of the training matrix with the imputed values

In [5]:
# Perform SVD on matrix A
u, s, vh = np.linalg.svd(A, full_matrices=False)
s_diag = np.diag(np.sqrt(s))
u_prime = np.dot(u, s_diag)
vh_prime = np.dot(vh,s_diag)

print('Shape of userValue matrix: %d x %d' % u_prime.shape)
print('Shape of itemValue matrix: %d x %d' % vh_prime.shape)

Shape of userValue matrix: 10000 x 1000
Shape of itemValue matrix: 1000 x 1000


** Model Selection **: Select a number k of eigenvalues to be used and truncate U and V accordingly. Evaluate the model performace.

In [6]:
# rms = []
# k_range = np.arange(1, 201, 10)

# for k in k_range:
#     A_pred = np.dot(u_prime[:, 0:k], vh_prime[0:k, :])
#     df_train['my_Prediction'] = A_pred[df_train['row_id']-1, df_train['col_id']-1].T
#     rms.append(RMS(df_train['my_Prediction'], df_train['Prediction']))
    
# plt.plot(k_range, rms)
# plt.show()

###  Step 3: Make Prediction 

Make prediction about the missing values with gradient descent algorithms

** Algorithm 1: Stocastic Gradient Descent **

The misfit function here is given as: 
$$ \min_{q,p}\left.\{ \sum_{u,i\in\kappa}{(r_{ui}-q_i^Tp_u)^2} + \lambda(||q_i||^2 + ||p_u||^2)\right.\} $$

In [7]:
def misfit_func(r, userValue, itemValue, lamda, df_train):
    global user_N, item_N
    
    r_pred = np.dot(userValue, itemValue)
    df_train['my_Prediction'] = r_pred[df_train['row_id']-1, df_train['col_id']-1].T
    err = df_train['Prediction'] - df_train['my_Prediction']
    loss = (err **2).sum()# + lamda * (norm(userValue, axis=1)+norm(itemValue, axis=0))
    err_matrix = coo_matrix((err, (df_train['row_id']-1, df_train['col_id']-1))) 
    return loss, err_matrix

def update_function(itemValue, userValue, lamda, err, gamma):
    
    global user_N, item_N
    
    paras = np.vstack((itemValue.T, userValue))
    
    coeff_matrix = bmat([[None, err.T], [err, None]]).toarray()
#     coeff_matrix[0:1000, 0:1000] = -lamda
#     coeff_matrix[1000:, 1000:] = -lamda
    
    new_paras = gamma * np.dot(coeff_matrix, paras) + paras
    
    new_itemValue = new_paras[0:item_N, :]
    new_userValue = new_paras[item_N:, :]
    
    return new_itemValue.T, new_userValue

In [8]:
k_select = 10      # Turncation number 
gamma = -0.001       # Learning rate
max_iter = 1000      # Maximum iteration
misfit = []          # store the misfit value
epsilon = 1e-3       # accept condition

lamda=10

In [11]:
i_iter = 1
tic = time()

userValue = u_prime[:, 0:k_select]
itemValue = vh_prime[0:k_select, :]
print(userValue.shape, itemValue.shape)

user_id = df_train['row_id']-1
item_id = df_train['col_id']-1

r = df_train['Prediction']


(10000, 10) (10, 1000)


In [14]:
loss, err = misfit_func(r, userValue, itemValue, lamda, df_train)
print('%.2e' % loss)
print(err.toarray().T.shape)
while (i_iter <= max_iter and loss > epsilon):
    
    itemValue, userValue = update_function(itemValue, userValue, lamda, err, gamma)
    loss, err = misfit_func(r, userValue, itemValue, lamda, df_train)
        
    if i_iter % 2 == 0:
        toc = time()
        print('Iteration: %d, Misfit: %.2e, Average time per iteration: %.4f' % 
              (i_iter, loss, (toc-tic)/i_iter))
    
    i_iter += 1

nan
(1000, 10000)


In [ ]:
print(userValue.shape, itemValue.shape)